In [1]:
import numpy
import scipy.optimize
import random
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [2]:
path = '/Users/dmarc/Desktop/School Work/DSC106/Homework/Final Project/beer_50000.json'
f = open(path, 'r')

In [3]:
dataset = []
for i in range(50000):
    # I had to use the eval function in this case because using the json loader
    # would just give me an error. 
    dataset.append(eval(f.readline()))

In [4]:
all_styles = []
all_users = []

for review in dataset:
    all_styles.append(review['beer/style'])
    review.update( {'type' : ''} )
    all_users.append(review['user/profileName'])
    review.update( {'review/length' : len(review['review/text'].split())})

In [5]:
# ADDING TYPE CLASSIFICATION
for review in dataset:
    if 'Ale' or 'IPA' in review['beer/style']:
        review.update(type = 0 )
    if 'Lager' in review['beer/style']:
        review.update(type = 1)
    if 'Stout' in review['beer/style']:
        review.update(type = 2)

In [6]:
lager = 0
stout = 0
ale = 0
lager_count = 2939
stout_count = 9713
ale_count = 37348
lager_review = 0
stout_review = 0
ale_review = 0
for review in dataset:
    if review['type'] == 'Lager':
        lager+=review['review/overall']
        lager_review += review['review/length']
    if review['type'] == 'Stout':
        stout+=review['review/overall']
        stout_review += review['review/length']
    if review['type'] == 'Ale':
        ale+=review['review/overall']
        ale_review += review['review/length']

In [7]:
# Unique words
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
#stemmer = PorterStemmer()
for d in dataset:
    r = ''.join([c for c in d['review/text'].lower() if not c in punctuation])
    for w in r.split():
       # w = stemmer.stem(w)
        wordCount[w] += 1

In [8]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]]

In [9]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [10]:
def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['review/text'].lower() if not c in punctuation])
    for w in r.split():
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

In [11]:
X = [feature(d) for d in dataset]

In [12]:
# ALE CLASSIFIER
y_ale = [d['type'] == 0 for d in dataset]
clf = LogisticRegression(class_weight='balanced')
clf.fit(X,y_ale)

C:\Users\dmarc\Anaconda3.7\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [13]:
theta = clf.coef_[0]
weights_ale = list(zip(theta, words + ['constant_feat']))
weights_ale.sort()
weights_ale.reverse()
weights_ale

[(3.2367079142701374, 'porters'),
 (2.6112353384117744, 'ipa'),
 (2.475599204291666, 'porter'),
 (2.240324484016824, 'dipa'),
 (2.071261154161332, 'apa'),
 (2.0152984450490177, 'barleywine'),
 (1.8351355671434313, 'clove'),
 (1.6261855056146806, 'ales'),
 (1.6024377241728016, 'grapefruit'),
 (1.476058729505242, 'rye'),
 (1.3870224383560856, 'ale'),
 (1.35460842433735, 'murky'),
 (1.296101397140019, 'pink'),
 (1.2184544737448266, 'scotch'),
 (1.2110754840693865, 'bastard'),
 (1.2086185131200513, 'ipas'),
 (1.1562338886556351, 'belgian'),
 (1.1180675605866772, 'campfire'),
 (1.1176748196582842, 'copper'),
 (1.1156987568244585, 'cloudy'),
 (1.0962306308173622, 'mahogany'),
 (1.0805735196991422, 'piney'),
 (1.0447523500366327, 'raspberry'),
 (1.0033432252150005, 'meat'),
 (0.9614820792700846, 'resinous'),
 (0.9363037873473219, 'nutmeg'),
 (0.8872902635448539, 'amber'),
 (0.879819851098977, 'pils'),
 (0.8760789013533299, 'hazy'),
 (0.8701558789014038, 'citrusy'),
 (0.8574583171378453, 'pump

In [14]:
# LAGER CLASSIFIER
y_lager = [d['type'] == 1 for d in dataset]
clf = LogisticRegression(class_weight='balanced')
clf.fit(X,y_lager)

C:\Users\dmarc\Anaconda3.7\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [15]:
theta = clf.coef_[0]
weights_lager = list(zip(theta, words + ['constant_feat']))
weights_lager.sort()
weights_lager.reverse()
weights_lager

[(2.4605043002711566, 'lager'),
 (1.3264071214050734, '500ml'),
 (1.253867992032387, 'gold'),
 (1.1764830926043748, 'keeps'),
 (1.0626793181255727, 'yellow'),
 (0.8551528931145249, 'water'),
 (0.8549238374820461, 'straw'),
 (0.826681213446589, 'during'),
 (0.8175382332107429, 'barrel'),
 (0.8112520946670053, 'cap'),
 (0.7476780030244831, 'surprise'),
 (0.7086681967829244, 'golden'),
 (0.7016930045939641, 'lasts'),
 (0.6955041446376908, 'lets'),
 (0.6920591496910546, 'three'),
 (0.6859550636582723, 'cold'),
 (0.6760496396447419, 'candy'),
 (0.6702925538072614, 'smokiness'),
 (0.6698630909568155, 'mildly'),
 (0.6695250731941006, 'finishing'),
 (0.6372586854181514, 'help'),
 (0.628975159542705, 'perfectly'),
 (0.6211671535019662, 'mind'),
 (0.6205708693493991, 'bland'),
 (0.620212262769903, 'aspect'),
 (0.616167394126075, 'between'),
 (0.6136010814254339, 'corn'),
 (0.6099387127817201, 'grain'),
 (0.6020197643977826, 'own'),
 (0.5991595151129531, 'fingers'),
 (0.5901406905237746, 'place')

In [16]:
# STOUT CLASSIFIER
y_stout = [d['type'] == 2 for d in dataset]
clf = LogisticRegression(class_weight='balanced')
clf.fit(X,y_stout)

C:\Users\dmarc\Anaconda3.7\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [17]:
theta = clf.coef_[0]
weights_stout = list(zip(theta, words + ['constant_feat']))
weights_stout.sort()
weights_stout.reverse()
weights_stout

[(3.0779860175559315, 'ris'),
 (2.88604811096882, 'stouts'),
 (2.5169525380635975, 'stout'),
 (2.262377043517512, 'kbs'),
 (2.188497806452728, 'oats'),
 (1.8466676200217391, 'oil'),
 (1.6862208199617839, 'boris'),
 (1.6546522489407258, 'oatmeal'),
 (1.5235896732042749, 'hype'),
 (1.2178703891686564, 'black'),
 (1.074012437877456, 'coffee'),
 (1.0531762641250193, 'coating'),
 (1.049678600859247, 'viscous'),
 (0.9691955148382823, 'espresso'),
 (0.9308649856370992, 'tobacco'),
 (0.887755162039127, 'exceptional'),
 (0.8757290179046402, 'pitch'),
 (0.8681216008762314, '22'),
 (0.8675593608435906, 'ones'),
 (0.8172180324920645, 'jet'),
 (0.8114234479101913, 'tell'),
 (0.8034235279108561, '22oz'),
 (0.7835058802363405, 'imperial'),
 (0.7406123955919734, 'roastiness'),
 (0.7292684719450572, 'regular'),
 (0.684301467845096, 'collar'),
 (0.6650851723147652, 'peppery'),
 (0.661068141917454, 'aging'),
 (0.6468408145009501, 'chocolate'),
 (0.6318879936453907, 'sense'),
 (0.6234129124859156, 'bomber

In [18]:
stout_rating = []
lager_rating = []
ale_rating = []
for review in dataset:
    if review['type'] == 2:
        stout_rating.append(review['review/overall'])
    if review['type'] == 1:
        lager_rating.append(review['review/overall'])
    if review['type'] == 0:
        ale_rating.append(review['review/overall'])

In [19]:
dataset[0]

{'review/appearance': 2.5,
 'beer/style': 'Hefeweizen',
 'review/palate': 1.5,
 'review/taste': 1.5,
 'beer/name': 'Sausa Weizen',
 'review/timeUnix': 1234817823,
 'beer/ABV': 5.0,
 'beer/beerId': '47986',
 'beer/brewerId': '10325',
 'review/timeStruct': {'isdst': 0,
  'mday': 16,
  'hour': 20,
  'min': 57,
  'sec': 3,
  'mon': 2,
  'year': 2009,
  'yday': 47,
  'wday': 0},
 'review/overall': 1.5,
 'review/text': 'A lot of foam. But a lot.\tIn the smell some banana, and then lactic and tart. Not a good start.\tQuite dark orange in color, with a lively carbonation (now visible, under the foam).\tAgain tending to lactic sourness.\tSame for the taste. With some yeast and banana.',
 'user/profileName': 'stcules',
 'review/aroma': 2.0,
 'type': 0,
 'review/length': 49}

In [20]:
types = {0: 'ale', 1: 'lager', 2: 'stout'}

In [21]:
ale_sentiments = defaultdict(int)
for review in ale_rating: 
    ale_sentiments[review] += 1

stout_sentiments = defaultdict(int)
for review in stout_rating: 
    stout_sentiments[review] += 1

lager_sentiments = defaultdict(int)
for review in ale_rating: 
    lager_sentiments[review] += 1

In [22]:
len(weights_ale)

1001

In [23]:
weights_lager

[(2.4605043002711566, 'lager'),
 (1.3264071214050734, '500ml'),
 (1.253867992032387, 'gold'),
 (1.1764830926043748, 'keeps'),
 (1.0626793181255727, 'yellow'),
 (0.8551528931145249, 'water'),
 (0.8549238374820461, 'straw'),
 (0.826681213446589, 'during'),
 (0.8175382332107429, 'barrel'),
 (0.8112520946670053, 'cap'),
 (0.7476780030244831, 'surprise'),
 (0.7086681967829244, 'golden'),
 (0.7016930045939641, 'lasts'),
 (0.6955041446376908, 'lets'),
 (0.6920591496910546, 'three'),
 (0.6859550636582723, 'cold'),
 (0.6760496396447419, 'candy'),
 (0.6702925538072614, 'smokiness'),
 (0.6698630909568155, 'mildly'),
 (0.6695250731941006, 'finishing'),
 (0.6372586854181514, 'help'),
 (0.628975159542705, 'perfectly'),
 (0.6211671535019662, 'mind'),
 (0.6205708693493991, 'bland'),
 (0.620212262769903, 'aspect'),
 (0.616167394126075, 'between'),
 (0.6136010814254339, 'corn'),
 (0.6099387127817201, 'grain'),
 (0.6020197643977826, 'own'),
 (0.5991595151129531, 'fingers'),
 (0.5901406905237746, 'place')

In [24]:
weights_stout

[(3.0779860175559315, 'ris'),
 (2.88604811096882, 'stouts'),
 (2.5169525380635975, 'stout'),
 (2.262377043517512, 'kbs'),
 (2.188497806452728, 'oats'),
 (1.8466676200217391, 'oil'),
 (1.6862208199617839, 'boris'),
 (1.6546522489407258, 'oatmeal'),
 (1.5235896732042749, 'hype'),
 (1.2178703891686564, 'black'),
 (1.074012437877456, 'coffee'),
 (1.0531762641250193, 'coating'),
 (1.049678600859247, 'viscous'),
 (0.9691955148382823, 'espresso'),
 (0.9308649856370992, 'tobacco'),
 (0.887755162039127, 'exceptional'),
 (0.8757290179046402, 'pitch'),
 (0.8681216008762314, '22'),
 (0.8675593608435906, 'ones'),
 (0.8172180324920645, 'jet'),
 (0.8114234479101913, 'tell'),
 (0.8034235279108561, '22oz'),
 (0.7835058802363405, 'imperial'),
 (0.7406123955919734, 'roastiness'),
 (0.7292684719450572, 'regular'),
 (0.684301467845096, 'collar'),
 (0.6650851723147652, 'peppery'),
 (0.661068141917454, 'aging'),
 (0.6468408145009501, 'chocolate'),
 (0.6318879936453907, 'sense'),
 (0.6234129124859156, 'bomber

In [29]:
list_ale = [list(tuplez) for tuplez in weights_ale]
list_lager = [list(tuplez) for tuplez in weights_lager]
list_stouts = [list(tuplez) for tuplez in weights_stout]

# Normalize each element[0] in the list
for element in list_ale:
    element[0] = int(element[0] * 100 )
    
for element in list_lager:
    element[0] = int(element[0] * 100)
    
for element in list_stouts:
    element[0] = int(element[0] * 100)

In [52]:
#list_ale
ale_counts = [d[0] for d in list_ale]
ale_words = [d[1] for d in list_ale]

print(ale_counts[:25])

[323, 261, 247, 224, 207, 201, 183, 162, 160, 147, 138, 135, 129, 121, 121, 120, 115, 111, 111, 111, 109, 108, 104, 100, 96]


In [57]:
lager_counts = [d[0] for d in list_lager]
lager_words = [d[1] for d in list_lager]

lager_counts[:25]

[246,
 132,
 125,
 117,
 106,
 85,
 85,
 82,
 81,
 81,
 74,
 70,
 70,
 69,
 69,
 68,
 67,
 67,
 66,
 66,
 63,
 62,
 62,
 62,
 62]

In [60]:
stout_count = [d[0] for d in list_stouts]
stout_words = [d[1] for d in list_stouts]

stout_count[:25]

[307,
 288,
 251,
 226,
 218,
 184,
 168,
 165,
 152,
 121,
 107,
 105,
 104,
 96,
 93,
 88,
 87,
 86,
 86,
 81,
 81,
 80,
 78,
 74,
 72]

In [49]:
import csv

csvfile = '/Users/dmarc/Desktop/School Work/DSC106/Homework/Final Project/wordcloud_ale.txt'

#Assuming res is a flat list
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator=',\n')
    for idx in range(len(ale_words)):
        writer.writerow([ale_words[idx]])
        writer.writerow([ale_count[idx]])


TypeError: 'int' object is not subscriptable

In [19]:
print(stout_rating)

[3.0, 3.0, 2.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.5, 4.5, 3.5, 4.0, 4.5, 4.5, 5.0, 5.0, 4.0, 4.5, 4.0, 4.0, 4.5, 4.5, 3.0, 3.5, 3.5, 4.0, 4.0, 4.5, 4.5, 5.0, 4.0, 3.5, 3.0, 4.5, 4.5, 4.0, 4.0, 3.0, 4.0, 4.5, 3.5, 2.5, 4.5, 4.0, 4.5, 4.0, 3.5, 4.5, 4.5, 4.0, 5.0, 4.0, 5.0, 4.0, 4.5, 3.0, 4.0, 4.0, 4.0, 3.5, 4.0, 4.0, 4.5, 3.5, 4.0, 3.0, 3.5, 4.0, 3.0, 3.5, 4.0, 3.5, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.5, 4.0, 4.0, 3.5, 3.5, 2.5, 4.0, 4.5, 2.5, 3.0, 3.5, 3.5, 4.0, 4.5, 4.0, 3.5, 4.0, 4.0, 4.0, 4.5, 3.0, 3.0, 3.0, 3.5, 4.0, 5.0, 3.5, 3.0, 3.5, 4.5, 3.5, 4.5, 4.0, 4.0, 4.0, 4.5, 4.0, 4.5, 4.5, 4.5, 3.0, 4.0, 4.5, 4.5, 4.5, 3.5, 3.5, 3.0, 4.0, 4.5, 3.0, 3.0, 4.0, 4.5, 3.5, 4.0, 5.0, 3.5, 3.0, 4.0, 4.0, 4.5, 4.0, 4.0, 4.5, 4.5, 3.5, 3.5, 4.0, 5.0, 4.5, 3.5, 4.0, 4.0, 4.0, 4.5, 4.5, 4.0, 4.0, 3.5, 4.5, 4.0, 4.0, 4.5, 4.0, 4.5, 4.0, 4.5, 2.5, 3.5, 2.5, 4.0, 4.0, 4.0, 4.0, 3.5, 4.0, 4.5, 4.0, 4.5, 3.0, 4.5, 4.5, 4.0, 4.5, 4.0, 5.0, 3.5, 3.5, 4.0, 4.5, 4.0, 4.5, 4.0, 5.0, 4.0, 4.5, 4.5,

In [20]:
import csv

csvfile = "/Users/alexkim/Desktop/stout_rating.txt"
#Assuming res is a flat list
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator=',\n')
    for val in stout_rating:
        writer.writerow([val])
    for val in lager_rating:
        writer.writerow([val])
    for val in ale_rating:
        writer.writerow([val])

FileNotFoundError: [Errno 2] No such file or directory: '/Users/alexkim/Desktop/stout_rating.txt'

In [ ]:
csvfile = "/Users/alexkim/Desktop/lager_rating.txt"
#Assuming res is a flat list
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator=',\n')
    for val in lager_rating:
        writer.writerow([val])  

In [ ]:
csvfile = "/Users/alexkim/Desktop/ale_rating.txt"
#Assuming res is a flat list
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator=',\n')
    for val in ale_rating:
        writer.writerow([val])  

In [21]:
# Word Clouds #
dataset[0]

{'review/appearance': 2.5,
 'beer/style': 'Hefeweizen',
 'review/palate': 1.5,
 'review/taste': 1.5,
 'beer/name': 'Sausa Weizen',
 'review/timeUnix': 1234817823,
 'beer/ABV': 5.0,
 'beer/beerId': '47986',
 'beer/brewerId': '10325',
 'review/timeStruct': {'isdst': 0,
  'mday': 16,
  'hour': 20,
  'min': 57,
  'sec': 3,
  'mon': 2,
  'year': 2009,
  'yday': 47,
  'wday': 0},
 'review/overall': 1.5,
 'review/text': 'A lot of foam. But a lot.\tIn the smell some banana, and then lactic and tart. Not a good start.\tQuite dark orange in color, with a lively carbonation (now visible, under the foam).\tAgain tending to lactic sourness.\tSame for the taste. With some yeast and banana.',
 'user/profileName': 'stcules',
 'review/aroma': 2.0,
 'type': 0,
 'review/length': 49}